In [15]:
from music21 import *
from comp2 import *

In [16]:
melody_in: stream.Stream = parse_midi("example_inputs/finalcountdown.mid")

print("Original melody:")
melody_in.show('midi');

Original melody:


In [17]:
print("Simple triads:")
simple_triads = get_triads(melody_in)

Simple triads:
['v', 'i', 'iv', 'iv', 'III', 'i', 'iv', 'iv', 'III', 'i', 'iv', 'iv', 'III', 'iv', 'v', 'v']


In [18]:
out_score = stream.Score()
out_score.insert(0, melody_in)
out_score.insert(0, simple_triads)

out_score.show('midi')

In [20]:
markov_table = generate_markov_table()
substituted_chords = substitute_chords(simple_triads, markov_table)
for i in range(5):
    substituted_chords = substitute_chords(substituted_chords, markov_table)

out_score = stream.Score()
out_score.insert(0, melody_in)
out_score.insert(0, substituted_chords)

out_score.show('midi')

Total number of states: 2744

new chords:
['v2', 'i+6', 'VI7', 'iii', 'III', 'i2', 'iv2', 'vii', 'III2', 'i2', 'iv+6', 'vii+6', 'III2', 'iv', 'v2', 'vii']

new chords:
['v7', 'IV7', 'VI7+6', 'iii', 'III', 'VI2', 'iv+6', 'vii2', 'III2', 'i2', 'VII2', 'V7', 'III2', 'iv2', 'vii+6', 'i']

new chords:
['v7', 'i2', 'VII', 'i2', 'III', 'VI7sus4', 'II', 'vi', 'III2', 'i', 'VII', 'ii', 'v2', 'VI2', 'V7', 'i']

new chords:
['v2', 'i+6', 'VII', 'III', 'III2', 'v2', 'v2', 'vii', 'III', 'VI2', 'VII', 'vii2', 'v7', 'i', 'ii2', 'v7']

new chords:
['v2', 'IV2', 'VII2', 'i', 'VI2', 'VI7sus4', 'v2', 'i', 'III', 'VI2', 'iv+6', 'III2', 'v2', 'i2', 'III', 'v2']

new chords:
['v7', 'i', 'II+6', 'III', 'VI7', 'v7', 'iv2', 'i2', 'iv2', 'VI7', 'VII2', 'III2', 'VI2', 'i2', 'II+6', 'v2']
